# Generalised Bernoulli map with various number formats

Revisiting BM Boghosian, PV Coveney, and H Wang. *A new pathology in the Simulation of Chaotic Dynamical Systems on Digital Computers*, **Adv. Theory Simul.** 2019, 2, 1900125, DOI: 10.1002/adts.201900125

### 0. Load packages

In [38]:
using PyPlot, FileIO, JLD2, Printf
using StochasticRounding, SoftPosit, LogFixPoint16s
using StatsBase, LinearAlgebra, Statistics
LogFixPoint16s.set_nfrac(10)   # use the 16-bit logfix format similar to Float16

┌ Warning: LogFixPoint16 was changed to 5 integer and 10 fraction bits.
└ @ LogFixPoint16s /Users/milan/.julia/packages/LogFixPoint16s/TGYbV/src/change_format.jl:25


### 1. Functions
Define the generalised Bernoulli map

$\quad x_{j+1} = f_\beta(x_j) = \beta x_j \mod 1$

with $\beta > 1$.

In [14]:
function bernoulli_orbit( x::T,            # initial condition
                          β::T,            # Bernoulli parameter
                          N0::Int,         # spin up iterations
                          N::Int) where T  # max period length
    oone = one(T)                  # 1 in format T
    
    # SPIN UP
    for i in 1:N0
        x = β*x
        x = x >= oone ? x-oone : x # x mod 1
    end
    
    # CHECK FOR PERIOD LENGTH
    x0 = x                         # new initial condition
    n = 0                          # orbit length (0 = not found yet)
    j = 0                          # iteration counter
    
    while n == 0 && j < N
        j += 1
        x = β*x
        x = x >= oone ? x-oone : x # x mod 1
        n = x0 == x ? j : 0        # check for periodicity
    end

    return n,x
end

bernoulli_orbit (generic function with 2 methods)

In [3]:
function bernoulli_map(x::T,β::T,N::Int) where T
    oone = one(T)                  # 1 in format T
    xout = Array{T,1}(undef,N+1)   # preallocate
    xout[1] = x                    # store initial condition
    for i in 2:N+1
        x = β*x
        while x >= oone            # x mod 1
            x = x - oone
        end
        xout[i] = x                # store iteration
    end
    return xout
end

bernoulli_map (generic function with 1 method)

In [70]:
function find_orbits(::Type{T},β::Real;
                    n::Int=100000,N0::Int=50000,Nmax::Int=1000000) where T

    println("β = $β, $T")
    
    O = fill(0,n)
    X = Array{T}(undef,n)

    for i in 1:n
        O[i],X[i] = bernoulli_orbit(T(rand()),T(β),50000,1000000)
    end

    orbit_lengths = sort([o for o in Set(O)])

    for (iol,ol) in enumerate(orbit_lengths)

        Xini = X[O .== ol]
        S = Set{T}()

        for x in Xini
            xorbit = bernoulli_map(x,T(β),ol)
            S = S ∪ Set(minimum(xorbit))
        end
    
        p = length(Xini)/n*100
        x0 = [s for s in S][1]
        
        s1 = "Orbit $iol ($(length(S))x): length = $(@sprintf("%8d",ol)), "
        s2 = "x₀ = $(@sprintf("%.14f",x0)), "
        s3 = "p = $(@sprintf("%5.2f",p))%"
        
        println(s1*s2*s3)
    end
end

find_orbits (generic function with 2 methods)

In [74]:
find_orbits(Float32,9//8,n=100000)

β = 9//8, Float32
Orbit 1 (1x): length =       22, x₀ = 0.08099699020386, p =  0.00%
Orbit 2 (1x): length =       78, x₀ = 0.00321817398071, p =  0.00%
Orbit 3 (1x): length =     1013, x₀ = 0.00107347965240, p =  3.01%
Orbit 4 (1x): length =     5258, x₀ = 0.00027680397034, p = 96.99%
